# Загрузка и подготовка данных

Источники данных

1. [Yahoo finance](https://finance.yahoo.com/)

Котировки акций, цена на нефть Brent на NY

2. [ЦБ РФ - Значения кривой бескупонной доходности государственных облигаций (% годовых)](http://www.cbr.ru/hd_base/zcyc_params/?UniDbQuery.Posted=True&UniDbQuery.From=01.01.2016&UniDbQuery.To=01.01.2022)

[MOEX](https://www.moex.com/ru/marketdata/indices/state/g-curve/archive/)
Процентные ставки на разные сроки

3. [RUSBONDS](https://rusbonds.ru/filters/bonds/new)
Поиск облигаций

[Tinkoff invest API](https://tinkoff.github.io/investAPI/)
Котировки облигаций

| ОФЗ         | ISIN         | ticker       | figi         | url |
| ----------- | -----------  | -----------  | -----------  | --- |
| ОФЗ 26207   | RU000A0JS3W6 | SU26207RMFS9 | BBG002PD3452 | https://www.tinkoff.ru/invest/bonds/SU26207RMFS9/coupons/ |
| ОФЗ 26209   | RU000A0JSMA2 | SU26209RMFS5 | BBG00386NQK6 | https://www.tinkoff.ru/invest/bonds/SU26209RMFS5/coupons/ |
| ОФЗ 26218   | RU000A0JVW48 | SU26218RMFS6 | BBG00B9PJ7V0 | https://www.tinkoff.ru/invest/bonds/SU26218RMFS6/coupons/ |
| ОФЗ 26211   | RU000A0JTJL3 | SU26211RMFS1 | BBG003TTSBB1 | https://www.tinkoff.ru/invest/bonds/SU26211RMFS1/coupons/ |
| ОФЗ 26212   | RU000A0JTK38 | SU26212RMFS9 | BBG00425VG07 | https://www.tinkoff.ru/invest/bonds/SU26212RMFS9/coupons/ |

4. [ЦБ РФ - цены на драгоценные металлы](https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted=True&UniDbQuery.From=01.01.2015&UniDbQuery.To=26.03.2022&UniDbQuery.Gold=true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium=true&UniDbQuery.so=1)

5. [ЦБ ФР - динамика курса валют](https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ=R01775&UniDbQuery.From=19.03.2022&UniDbQuery.To=26.03.2022)

6. [MOEX - RTSI](https://www.moex.com/ru/index/RTSI/archive/)


In [32]:
import datetime
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
import requests
import tinvest
import yfinance as yf
from bs4 import BeautifulSoup
from pytz import timezone
from tqdm.notebook import tqdm

In [90]:
begin = "2016-01-01"
end = "2022-01-01"

begin_ = datetime.datetime.strptime(begin, "%Y-%m-%d")
end_ = datetime.datetime.strptime(end, "%Y-%m-%d")

date_from = "01.01.2016"
date_to = "01.01.2022"

In [34]:
queries = {
    "FESH.ME": "FESH",
    "APTK.ME": "APTK",
    "RASP.ME": "RASP",
    "NKNCP.ME": "NKNCP",
    "MVID.ME": "MVID",
    "BELU.ME": "BELU",
    "PHOR.ME": "PHOR",
    "SNGSP.ME": "SNGSP",
    "SELG.ME": "SELG",
    "PIKK.ME": "PIKK",
    "YNDX.ME": "YNDX",
    "IMOEX.ME": "IMOEX",
    "BZ=F": "BRENT",
}

In [40]:
path = "./data/{}.csv"
for stock, out in tqdm(queries.items()):
    spath = path.format(out)
    if os.path.exists(spath):
        continue
    df = yf.download(stock, begin, end)
    df[out] = df["Adj Close"]  # .map(lambda x: )
    df[[out]].to_csv(spath)
    # df[out].plot.hist()
    # plt.show()

  0%|          | 0/13 [00:00<?, ?it/s]

In [48]:
with open(os.path.expanduser("~/.tcs_token"), "r") as f:
    TOKEN = f.read().strip()
tz = timezone("Europe/Moscow")
client = tinvest.SyncClient(TOKEN, use_sandbox=True)
bonds = client.get_market_bonds().payload.instruments
bonds = pd.DataFrame([json.loads(x.json()) for x in bonds])

In [49]:
bond_list = ["RU000A0JS3W6", "RU000A0JSMA2", "RU000A0JVW48", "RU000A0JTJL3", "RU000A0JTK38"]
bonds[bonds["isin"].isin(bond_list)]

,currency,figi,isin,lot,min_price_increment,name,ticker,type,min_quantity
102,RUB,BBG00B9PJ7V0,RU000A0JVW48,1,0.01,ОФЗ 26218,SU26218RMFS6,Bond,NaN
131,RUB,BBG002PD3452,RU000A0JS3W6,1,0.01,ОФЗ 26207,SU26207RMFS9,Bond,NaN
149,RUB,BBG00425VG07,RU000A0JTK38,1,0.01,ОФЗ 26212,SU26212RMFS9,Bond,NaN
383,RUB,BBG003TTSBB1,RU000A0JTJL3,1,0.01,ОФЗ 26211,SU26211RMFS1,Bond,NaN
602,RUB,BBG00386NQK6,RU000A0JSMA2,1,0.01,ОФЗ 26209,SU26209RMFS5,Bond,NaN


In [53]:
def get_max_duration(interval):
    if interval == tinvest.schemas.CandleResolution.min1:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min2:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min3:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min5:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min10:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min15:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.min30:
        return datetime.timedelta(days=1)
    elif interval == tinvest.schemas.CandleResolution.hour:
        return datetime.timedelta(days=7)
    elif interval == tinvest.schemas.CandleResolution.day:
        return datetime.timedelta(days=365)
    elif interval == tinvest.schemas.CandleResolution.week:
        return datetime.timedelta(days=365 * 2)
    elif interval == tinvest.schemas.CandleResolution.month:
        return datetime.timedelta(days=365 * 10)
    raise ValueError("unknown interval value {}".format(interval))


def get_candles(client, figi, from_=None, to_=None, *, interval):
    if to_ is None:
        to_ = datetime.now(tz=timezone("Europe/Moscow"))
    result = []
    duration = get_max_duration(interval)
    d2 = to_
    d1 = d2 - duration
    if from_ is not None and d1 < from_:
        d1 = from_
    while True:
        r = client.get_market_candles(figi, d1, d2, interval)
        result.extend(r.payload.candles)
        d2 = d1
        d1 -= duration
        if (from_ is not None and d2 < from_) or len(r.payload.candles) == 0:
            break
    result = pd.DataFrame([json.loads(x.json()) for x in result])
    result["time"] = pd.to_datetime(result["time"]).dt.tz_convert(tz)
    result = result.sort_values("time").reset_index(drop=True)
    result["Date"] = pd.to_datetime(result["time"].dt.date)
    result = result.set_index("Date")
    return result[["c"]]

In [55]:
bonds.head()

,currency,figi,isin,lot,min_price_increment,name,ticker,type,min_quantity
0,RUB,BBG00T22WKV5,RU000A101KT1,1,0.01,ОФЗ 29013,SU29013RMFS8,Bond,NaN
1,RUB,BBG00R05JT04,RU000A1013Y3,1,0.10,Черкизово выпуск 2,RU000A1013Y3,Bond,NaN
2,RUB,BBG00KHGQP89,RU000A0ZZ1F6,1,0.10,КарМани БО выпуск 2,RU000A0ZZ1F6,Bond,NaN
3,RUB,BBG00Q9K64Q5,RU000A100TD8,1,0.10,Солид-Лизинг выпуск 5,RU000A100TD8,Bond,NaN
4,RUB,BBG012NW2KW6,RU000A103QL1,1,0.10,Росагролизинг выпуск 3,RU000A103QL1,Bond,NaN


In [88]:
# ОФЗ
path = "./data/{}.csv"
for _, row in bonds[bonds["isin"].isin(bond_list)].iterrows():
    out = row['name'].replace('ОФЗ ', 'OFZ_')
    spath = path.format(out)
    if os.path.exists(spath):
        continue
    df = get_candles(client, row["figi"], begin_, end_, interval=tinvest.schemas.CandleResolution.day)
    df.rename({'c': out}, axis=1).to_csv(spath)

In [91]:
# Процентные ставки на разные сроки
url = "http://www.cbr.ru/hd_base/zcyc_params/"
page = requests.get(
    url,
    params={
        "UniDbQuery.Posted": "True",
        "UniDbQuery.From": date_from,
        "UniDbQuery.To": date_to,
    },
)
soup = BeautifulSoup(page.content, "html.parser")
res = soup.find("table", {"class": "data"})

df = pd.read_html(str(res).replace(",", "."))[0]
df = df.droplevel(0, "columns").rename(columns={"Дата": "Date"})
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
df = df.set_index("Date").astype(float)
df.columns = [float(x.replace(",", ".")) for x in df.columns]
df /= 100
df.to_csv("./data/cbr_g_curve.csv")

In [98]:
# Золото
url = "https://www.cbr.ru/hd_base/metall/metall_base_new"
page = requests.get(
    url,
    params={
        "UniDbQuery.Posted": "True",
        "UniDbQuery.From": date_from,
        "UniDbQuery.To": date_to,
        "UniDbQuery.Gold": True,
    },
)
soup = BeautifulSoup(page.content, "html.parser")
res = soup.find("table", {"class": "data"})
df = pd.read_html(str(res).replace(",", "."))[0]
df = df.rename(columns={"Дата* ▼": "Date", "Золото": "GOLD"})
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
df = df.set_index("Date")
df["GOLD"] = df["GOLD"].map(lambda x: float(x.replace(" ", "")))
df.to_csv('data/GOLD.csv')

In [111]:
# Доллар, франк
url = "https://www.cbr.ru/currency_base/dynamics/"
currencies = [('USD', 'R01235'), ('CHF', 'R01775')]
for curr, param in currencies:
    page = requests.get(
        url,
        params={
            "UniDbQuery.Posted": "True",
            "UniDbQuery.From": date_from,
            "UniDbQuery.To": date_to,
            "UniDbQuery.Gold": True,
            "UniDbQuery.VAL_NM_RQ": param,
        },
    )
    soup = BeautifulSoup(page.content, "html.parser")
    res = soup.find("table", {"class": "data"})
    df = pd.read_html(str(res).replace(",", "."), header=1)[0]
    df = df.rename(columns={"Дата ▼": "Date", "Курс": curr})
    df.drop(columns=['Единиц'], inplace=True)
    df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
    df = df.set_index("Date")
    # df[curr] = df[curr].map(lambda x: float(x.replace(" ", "").replace(',', '.')))
    df.to_csv(f"data/{curr}.csv")